# Constants: We prepare the constants for distortion calculations.

In [ ]:
# 79-character line limit
######### ######### ######### ######### ######### ######### ######### #########
%reset -f

import numpy as np # The NumPy library
#import math # The math module, np includes it
from scipy.integrate import quad # Method for integration in scipy.integrate sub-package


cells_num = 4

# The opitimized spaces for a Guassian distribution in a uniform quantizer(UQ).
# The sizes are for UQs for number of cells: 4, 8, 16, 32, 64, 128, 256.
all_size_gaus = np.array([0.9957, 0.5860, 0.3352, 0.1881, 0.1041, 0.0569, 0.0308])

cell_size = all_size_gaus[int(np.log2(cells_num))-2]

# np.arange has rounding error issue.
# boundaries_symm = np.arange(-(cells_num/2-1) * cell_size, # For symmetric distributions
#                              (cells_num/2) * cell_size,
#                             cell_size) 

boundaries_symm = np.linspace(-(cells_num/2-1) * cell_size, # For symmetric distributions
                               (cells_num/2-1) * cell_size,
                              cells_num-1)
n_inf = float("-inf")
p_inf = float("inf")

boundaries_symm = np.insert(boundaries_symm, 0, n_inf)
boundaries_symm = np.append(boundaries_symm, p_inf)

def pdf(x): # Defining the distribution
    gaus_std = 1
    gaus_mean = 0
    pdf = 1/(gaus_std*np.sqrt(2*np.pi)) * \
                  np.exp(-0.5*((x-gaus_mean)/gaus_std)**2) # Gaussian pdf
    return pdf

def xpdf(x):
    xpdf = x * pdf(x)
    return xpdf

def x2pdf (x):
    x2pdf = x * xpdf(x)
    return x2pdf

prbs = []
xprbs = []
x2prbs = []
cell_reps = []
for i in range (0, cells_num):
    cell_prb, integ_err = quad(pdf, boundaries_symm[i], boundaries_symm[i+1])
    prbs = np.append(prbs, cell_prb)
    
    cell_xprb, integ_err = quad(xpdf, boundaries_symm[i], boundaries_symm[i+1])
    xprbs = np.append(xprbs, cell_xprb)
    
    cell_x2prb, integ_err = quad(x2pdf, boundaries_symm[i], boundaries_symm[i+1])
    x2prbs = np.append(x2prbs, cell_x2prb)
    
    cell_rep = cell_xprb / cell_prb
    cell_reps = np.append(cell_reps, cell_rep)

# Keras Implementation

In [ ]:
# We use the Functional API option of Keras.

from keras import models
from keras import layers
from keras import Input
import keras.backend as K
import tensorflow as tf

l1_size = cells_num

input_tensor = Input(shape=(cells_num,))
l1_outs = layers.Dense(l1_size, activation = 'relu')(input_tensor)
print(l1_outs.shape)
l2_outs = layers.Dense(cells_num, activation = 'sigmoid')(l1_outs)

# def make_indexes(x):
#     raw_indexes = K.dot(x, cells_num)
#     indexes = K.round(raw_indexes)
#     for i in (0, cells_num-1):
#         for j in (i+1, cells_num):
#             if K.equal(indexes[i], indexes[j]):
                
#     return 

sorted_outs = layers.Lambda (sort_outs)(l2_outs)
#def distortion ()
